# Survey of Object Detection methods

This is the brief survey of major object detection methods up to date.


## Introduction

Deep learning has been applied to various application fields, and application scenarios are roughly divided into three categories: object recognition, target detection, and natural language processing.
This survey focuses on and analyzes the deep learning methods in the field of target detection, and conducts an in-depth analysis of the classic model framework.


Target detection can be understood as a combination of object recognition and object location, not only to identify which classification the object belongs to, but more importantly to obtain the specific location of the object in the image.


In order to accomplish these two tasks, the target detection model is divided into two categories. One type is two-stage, which divides object recognition and object positioning into two steps, which are respectively completed. The typical representatives of this class are R-CNN, fast R-CNN, faster-RCNN family and the modified/tweaked networks based on them. They have low recognition error rate and low recognition rate, but they are slow and cannot meet real-time detection scenarios.


In order to address this problem, another way has emerged, called one-stage, typical representatives are Yolo, SSD, YoloV2 and so on. They are fast-recognizing and can meet real-time requirements, and the accuracy is basically up to the level of the fastest R-CNN.



## Two-stage Detectors

## R-CNN

R-CNN method is invented in 2014, and it laid the application of the two-stage approach in the field of target detection. 

R-CNN's structure is as follows:
![title](img/r-cnn_arch.png)

The procedures are as follows:

1. Get the original image.

2. The selective search algorithm is used to evaluate the similarity between adjacent images, merge the images with high similarity, score the merged blocks, and select the candidate frame of the region of interest, that is, the subgraph. This step will have about approximately 2,000 subgraphs to be selected.

3. The convolutional neural network is separately used for the subgraph, and convolution-relu-pooling and full connection are performed to extract features. This step is basically the scope of object recognition.

4. The extracted features are classified into objects, and the blocks with high classification accuracy are retained as the final object positioning block.


R-CNN achieves a 50% performance improvement over the traditional target detection algorithm. With the VGG-16 model as the object recognition model, 66% accuracy can be achieved on the VOC2007 dataset, which is a very good result. The biggest drawback is that the process is relatively slow and the memory usage is very large. There are two main reasons:

1. The candidate box is completed by the traditional selective search algorithm, which is slower.

2. For 2000 candidate frames, object recognition is required, that is, 2000 convolutional network calculations are needed. This amount of calculation is very large.

## Fast R-CNN

In response to drawbacks of R-CNN, Microsoft proposed the fast R-CNN algorithm in 2015, which mainly focus on optimization of two problems.

Fast R-CNN algorithm's structure is as follows:
![title](img/fr-cnn_arch.png)

1. The *ROI pooling* pooling layer structure is proposed, which solves the problem that the candidate frame graph must be cropped and scaled to the same size. Since the input image size of the CNN network must be a fixed size (otherwise it cannot be calculated when fully connected), the candidate frames of different sizes and shapes in the R-CNN are cropped and scaled so that they reach the same size. This operation is a waste of time and easily leads to loss and deformation of image information. Fast R-CNN inserts the ROI pooling layer before the fully connected layer, so that the image does not need to be cropped, which solves this problem well.

For example, cropping can result in loss of information, and scaling can cause image distortion.
![title](img/crop_warp.png)

The idea of *ROI pooling* is that if we want to generate a picture of MxN, we first divide the feature map horizontally and vertically into M and N parts, then take the maximum value for each part and output the feature map of MxN. This achieves a fixed-size image output. The ROI pooling layer is located after the convolution layer, before the full connection layer.

2. The *multi-task loss function* idea is proposed, and the classification loss and the frame positioning regression loss are combined in training, and finally outputs the corresponding classification and frame coordinates.

## Faster R-CNN

One problem with both *R-CNN* and *fast R-CNN* is that candidate frames are generated by selective search. This algorithm is slow. Moreover, all of the 2000 or so candidate frames generated in the *R-CNN* need to go through a convolutional neural network, that is, the CNN network needs to pass about 2000 times. This is very time consuming (fast R-CNN has been improved, only Need to go through the CNN network for the whole picture). This is also the main reason for the slow detection speed of these two algorithms. In response to this problem, in the *Faster R-CNN*, the *RPN network* is used to obtain candidate frames, thus eliminating the selective search algorithm and requiring only one convolution layer operation, which greatly improves the recognition speed. This algorithm is very complicated and we will analyze it in detail. Its basic structure is as follows
![title](img/faster-rcnn.png)

Faster R-CNN are mainly composed of four major steps:

1. Convolution layer. The original image is first subjected to conv-relu-pooling multi-layer convolutional neural network to extract feature maps. For subsequent RPN networks and full connectivity layers. Faster R-CNN does not require convolutional feature extraction for each subgraph as R-CNN requires. It only needs to extract the entire image once, which greatly reduces the computation time.
2. RPN layer, *region proposal networks*. The RPN layer is used to generate a candidate frame, and uses softmax to determine whether the candidate frame is a foreground or a background, selects a foreground candidate frame from it (because the object is generally in the foreground), and uses the bounding box regression to adjust the position of the candidate frame to obtain a feature subgraph. Called proposals.
3. ROI layer, which is already mentioned in the *fast R-CNN*, it makes different size of the proposal pool into the same size, and then sent to the subsequent fully connected layer for object classification and position adjustment regression.
4. Classification layer. Using the feature map proposal output from the ROI layer, the category of the proposal is determined, and the bounding box is again regressed to obtain an accurate shape and position.

The network structure using the VGG-16 convolution model is as follows
![title](img/faster-rcnn_vgg16.png)

### Convolutional layer

The VGG-16 model used in the convolutional layer first scales the original image of PxQ into a MxN image, and then passes through 13 conv-relu layers, which interspersed with four max-pooling layers. All convolution kernels are 3x3, padding is 1, and stride is 1. The pooling layer kernel is 2x2, padding is 0, and stride is 2.

The picture of MxN, after convolution, becomes the feature map of (M/16) x (N/16).


### RPN layer

Faster R-CNN discards the *selective search* method in R-CNN, and uses the *RPN layer* to generate candidate frames, which can greatly improve the generation speed of candidate frames. The RPN layer first undergoes a 3x3 convolution operation and then splits into two paths. 

One path is to determine whether the candidate box is foreground or background. The candidate box is firstly reshaped into a one-dimensional vector, then softmax to determine whether it is foreground or background, and then reshape is restored to a two-dimensional feature map.

The other path is to determine the position of the candidate frame is achieved by bounding box regression.

After the two-way calculation is completed, the foreground candidate box is selected (because the object is in the foreground), and the calculated candidate frame position is used to obtain the feature sub-graph proposal that we are interested in.

#####  Candidate box generation anchors

The convolutional layer extracts the original image information and obtains 256 feature maps. After 3x3 convolution of the RPN layer, there are still 256 feature maps. But each point combines spatial information around 3x3. For each point on each feature map, generate k anchors (k defaults to 9). The anchor is divided into two categories: foreground and background (we don't care whether it is an airplane or a car, but only distinguish whether it is foreground or background). The anchor has four coordinate offsets [x, y, w, h], x, y represent the coordinates of the center point, and w and h represent the width and height. Thus, for each point on the feature map, k selection regions with different sizes and sizes are obtained.

#### softmax to decide foreground or background
For the generated anchors, we first need to determine whether it is foreground or background. Since the object of interest is in the foreground, after this step, we can discard the background anchors. Most of the anchors belong to the background, so this step can filter out many useless anchors, thus reducing the amount of computation of the fully connected layer.

For the 256 feature maps obtained after 3x3 convolution, the first 1x1 convolution is used to transform into 18 feature maps. Then reshape is a one-dimensional vector, which is judged by the softmax to be the foreground or the background. The only effect of reshape here is to make the data available for softmax calculations. Then output the identified foreground anchors.

#### Determine the candidate frame position
The other way is to determine the position of the candidate box, which is the [x, y, w, h] coordinate value of the anchors. As shown in the figure below, red represents our current constituency and green represents the real constituency. Although our current selection can roughly select the aircraft, but the actual position and shape of the green is still very different, it is necessary to adjust the generated anchors. This process is called bounding box regression.
![title](img/boundingboxloc.png)


Suppose the coordinates of the red box are [x, y, w, h], and the green box, that is, the coordinates of the target frame is [Gx, Gy, Gw, Gh], we want to create a transformation such that [x, y, w, h] can be changed to [Gx, Gy, Gw, Gh]. The simplest idea is to do the translation first so that the center point is close and then scale so that w and h are close. as follows
![title](img/dxdydwdh.png)

What we need to learn is the four transformations *dx dy dw dh*. Since it is a linear transformation, we can model it with linear regression. After setting the loss and optimization methods, deep learning networks can be used to train and get the model. For spatial position loss, we generally use the *mean square error* algorithm instead of *cross entropy* (cross entropy is used in classification prediction). The optimization method can use the *adaptive gradient descent algorithm* Adam.

#### Output feature submap proposal
After getting the foreground anchors and determining their position and shape, we can output the foreground feature map proposal. Proceed as follows

Get foreground anchors and their [x y w h] coordinates
According to the different probability of the anchors as the foreground, sort from large to small, select the pre_nms_topN anchors, such as the first 6000
Eliminate very small anchors
Through the NMS non-maximum suppression, the confidence is higher from the anchors. This is mainly to solve the problem of selecting overlaps. First calculate each selection area, then sort according to their score in softmax (that is, whether it is the probability of the foreground), and put the largest selection of the score into the queue. Next, calculate the IOU of the remaining selections and the current maximum score selection (IOU is the intersection of the two boxes divided by the area of ​​the two boxes, which measures the degree of overlap between the two boxes). Remove the selection where the IOU is greater than the set threshold. This solves the problem of overlapping areas.
Select the previous post_nms_topN results as the final selection propsal output, such as 300.
After this step, the object positioning should be basically finished, and the rest is the object recognition.

### ROI Pooling layer

Similar to fast R-CNN, this layer mainly solves the difference in the shape and size of the previous proposal, which makes it impossible to make a full connection. The full join calculation can only operate on the determined shape, so the proposal size must be made the same. This problem can be solved by cutting and scaling, but it will bring information loss and picture deformation. We use ROI pooling to solve this problem effectively.

In the ROI pooling, if the target output is MxN, the input proposal is divided into MxN shares in the horizontal and vertical directions, and each one takes a maximum value, thereby obtaining an output characteristic map of the MxN.

### Classification layer

The feature map behind the ROI Pooling layer, through the full connection layer and softmax, can calculate which specific category, such as people, dogs, airplanes, and can get the cls_prob probability vector. At the same time, using the bounding box regression to fine tune the proposal position, and get bbox_pred, which is used to return to the more accurate target detection frame.
![title](img/faster-rcnn-classification.png)


Faster R-CNN uses the resNet101 model as a convolutional layer, achieving an accuracy of 83.8% on the voc2012 dataset, exceeding yolo ssd and yoloV2. The biggest problem is that the speed is slow, and only 5 frames per second can be processed, which does not meet the real-time requirements.

## RetinaNet


## One-stage Detectors
Targeting at two-stage detectors common flaw of slow computation speed, one-stage detector is presented. And the major one-stage detectors are, SSD, YOLO.

## SSD

## DSSD

## YOLO: You Only Look Once

The idea behind R-CNN, Fast R-CNN is proposal + classification(proposal for location info, classification for class info). And they can archieve high accuracy while still processing speed is the shortcoming.

In view of the ubiquitous shortcomings of the two-stage target detection algorithms, YOLO creatively proposes one-stage: The object classification and object positioning are completed in one step. Yolo directly returns the bounding box location and the bounding box class at the output layer and the bounding box belongs to the one-stage. In this way, YOLO can achieve 45 frames per second of computing speed, fully meet the real-time requirements (if frame rate is up to 24 frames per second, the human eye will consider it as continuous). Its network structure is as follows
![title](img/yolov1_arch.png)

The three major layers of YOLO are: convolution layer, object detection layer and NMS layer.

### Convolution layer
Using the Google inceptionV1 network, mapping to the first stage in the above figure, a total of 20 layers. This layer is mainly for feature extraction, which improves the generalization ability of the model. However, the author modified InceptionV1. Instead of using the inception module structure, he replaced it with a 1x1 convolution and a 3x3 convolution in parallel. (to simplify the network structure)

### Object detection layer

The image passes through 4 convolutional layers and 2 fully connected layers, and finally produces 7x7x30 output. The purpose of passing through four convolutional layers is to improve the generalization of the model. Yolo splits a 448x448 original image into 7x7 grids. Each grid predicts the coordinates of the two bounding boxes (x, y, w, h) and the confidence of the objects contained in the box, and the objects belong to The probability of each of the 20 categories (yolo's training data is VOC2012, which is a 20-category data set). So the corresponding parameter for a grid is (4x2+2+20) = 30
![title](img/yolo-objectdet.png)

1. Bounding box coordinates: As shown above, each grid (red box) in the 7x7 grid corresponds to two bounding boxes (yellow boxes) with different sizes and shapes. The position coordinates of each box are (x, y, w, h), x and y represent the coordinates of the center point of the box, and w and h represent the width and height of the box. By comparing the actual coordinates (Gx, Gy, Gw, Gh) of the object calibrated on the training data set, the initial bounding box can be calculated and the final position of the model can be calculated.
2. Bounding box confidence: This confidence is only to express the probability of having objects in the box, and does not express what the objects in the box are.

$confidence=Pr(Object) * IOU^{truth}_{pred}$

The former item indicates that the object with or without manual marking falls into the grid, and if it is, it is 1, otherwise it is 0. The second term represents the degree of coincidence between the bounding box and the box of the real token. It is equal to the intersection of two box areas, divided by the area union. The larger the value, the closer the box is to the real position.

| Name | # Images (trainval) | # Classes | Last updated |
| --------------- | ------------------- | --------- | ------------ |
| ImageNet | 450k | 200 | 2015 |
| COCO | 120K | 90 | 2014 |
| Pascal VOC | 12k | 20 | 2012 |
| Oxford-IIIT Pet | 7K | 37 | 2012 |
| KITTI Vision | 7K | 3 | |

Each grid also needs to predict the probability that it belongs to each of the 20 categories. The classification information is for each grid, not for the bounding box. Therefore, only 20 categories, not 40. Confidence is for the bounding box. It only indicates whether there is an object in the box, and it does not need to predict which one of the 20 categories is the object, so only two parameters are needed. Although the classification information and confidence are both probabilities, the meanings are completely different.

### NMS layer

The filter layer is designed to filter out the most appropriate ones in multiple results (multiple bounding boxes), which is basically the same as in the faster R-CNN. It is first to filter out the box whose score is lower than the threshold, and perform NMS non-maximum suppression on the remaining boxes, and remove the box with higher overlap (the NMS specific algorithm can review the faster R-CNN section above). This gives you the final best fit for several boxes and their categories.

#### YOLO lost function

The loss function of yolo consists of three parts, position error, confidence error, and classification error. The specific formula is as follows
![title](img/yolo_calculation.jpg)


## yolo v2

## yolo v3

reference:
[1] https://blog.csdn.net/u012426298/article/details/80747361
